In [17]:
from dotenv import load_dotenv
from pinecone import Pinecone
from pinecone import Pinecone,ServerlessSpec

import os

load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
pine = Pinecone(api_key=PINECONE_API_KEY)

In [21]:
index_name='news'

In [22]:
pine.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

{
    "name": "news",
    "metric": "cosine",
    "host": "news-3h3vqd5.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()

c:\Users\C117\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [39]:
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 차원(dimension) 이 같아야 합니다.
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [32]:
from datasets import load_dataset
data = load_dataset("fancyzhx/ag_news",split='train[:100]')

In [33]:
print(type(data))
print(data)

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'label'],
    num_rows: 100
})


In [36]:
data[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [49]:
# 예: 숫자가 포함된 뉴스만 추출
candidates = [sample for sample in data if any(char.isdigit() for char in sample['text'])]

# 상위 5개 샘플 확인
for c in candidates[:5]:
    print(c)

{'text': 'Stocks End Up, But Near Year Lows (Reuters) Reuters - Stocks ended slightly higher on Friday\\but stayed near lows for the year as oil prices surged past  #36;46\\a barrel, offsetting a positive outlook from computer maker\\Dell Inc. (DELL.O)', 'label': 2}
{'text': "Money Funds Fell in Latest Week (AP) AP - Assets of the nation's retail money market mutual funds fell by  #36;1.17 billion in the latest week to  #36;849.98 trillion, the Investment Company Institute said Thursday.", 'label': 2}
{'text': 'Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.', 'label': 2}
{'text': "No Need for OPEC to Pump More-Iran Gov  TEHR

In [100]:
from uuid import uuid4

texts = data['text']
labels = data['label']
ids = [str(uuid4()) for _ in range(len(texts))]

# 메타데이터
metadatas = [{"label": l, "text": t} for l, t in zip(labels, texts)]

In [101]:
# 여러 문서 한 번에 임베딩
embeddings = embedding.embed_documents(texts)  # 리스트 형태 반환

In [43]:
print(type(embeddings))           # 리스트여야 함
print(type(embeddings[0]))        # 리스트 or numpy.ndarray
print(type(embeddings[0][0]))     # float 이어야 함

<class 'list'>
<class 'list'>
<class 'float'>


In [103]:
# Pinecone 업서트
index.upsert(
    vectors = [
    {
        "id": id_,
        "values": emb,
        "metadata": meta
    }
    for id_, emb, meta in zip(ids, embeddings, metadatas)
    ],
    namespace='news_ns1'
)

{'upserted_count': 100}

In [129]:
from pinecone import Pinecone

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [130]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "news"

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'news_ns1': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}

In [149]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 객체를 초기화합니다.
# 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 질문 리스트를 정의합니다.
question = ["By how much did the assets of U.S. retail money market mutual funds fall in the latest week?"]

# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)[0]

In [148]:
query_result = index.query(
    namespace="news_ns1",       # news-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,   # 임베딩된 질문 벡터를 사용합니다.
    top_k=5,                    # 상위 5개의 결과를 반환합니다.
    include_vector=False,       # 벡터를 포함하지 않습니다.
    include_metadata=True       # 메타데이터를 포함합니다.
)

In [147]:
from langchain_pinecone import PineconeVectorStore

# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "full_text"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="text")

# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="news_ns1")

In [138]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-4.1-mini')

In [ ]:
question = "최근 한 주 동안 미국 소매 머니 마켓 뮤추얼 펀드의 자산이 얼마나 감소했나요?"

chat_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 백과사전 처럼 여러가지 지식을 갖고 있습니다. 
                질문에 대해 {context} 활용하여 잘 설명해주세요.'''),
    ('user','''{question} 에 대하여 지식을 알려주세요.''')          
])

In [144]:
context = ""
for doc in docs:
    context += doc.page_content
print(context)

Money Funds Fell in Latest Week (AP) AP - Assets of the nation's retail money market mutual funds fell by  #36;1.17 billion in the latest week to  #36;849.98 trillion, the Investment Company Institute said Thursday.In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market these days, but investors in value-focused mutual funds have reason to feel a bit smug -- if only because they've lost less than the folks who stuck with growth.Dollar Falls Broadly on Record Trade Gap  NEW YORK (Reuters) - The dollar tumbled broadly on Friday  after data showing a record U.S. trade deficit in June cast  fresh doubts on the economy's recovery and its ability to draw  foreign capital to fund the growing gap.Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.Wall S

In [145]:
prompt = chat_template.format_messages(question=question,context=context)
response = llm.invoke(prompt)
print(response.content)

최근 한 주 동안 미국 소매 머니 마켓 뮤추얼 펀드의 자산은 11억 7천만 달러(약 1.17 billion 달러) 감소했습니다. 투자회사협회(Investment Company Institute)의 발표에 따르면, 이 기간 동안 소매 머니 마켓 뮤추얼 펀드 자산 총액은 약 849.98조 달러 수준으로 집계되었습니다. 이는 투자자들이 최근 시장 상황에서 자금을 인출하거나 다른 투자처로 이동시키면서 소매 머니 마켓 펀드의 자산이 줄어든 결과로 해석할 수 있습니다.


In [ ]:
# 랭체인 안쓸 때 , 처음 했던 방식
from openai import OpenAI

client = OpenAI()
question = "최근 한 주 동안 미국 소매 머니 마켓 뮤추얼 펀드의 자산이 얼마나 감소했나요?"
test = client.chat.completions.create(
    model = 'gpt-4.1-mini',
    messages = [
        {"role":"system","content":'''당신은 백과사전 처럼 여러가지 지식을 갖고 있습니다. 
                질문에 대해 {context} 활용하여 잘 설명해주세요.'''},
        {"role":"user","content":f'{question} 에 대하여 지식을 알려주세요.'}        
    ],
    temperature=0.1
)

print(test.choices[0].message.content)

최근 한 주 동안 미국 소매 머니 마켓 뮤추얼 펀드의 자산은 약 1,000억 달러 감소했습니다. 이는 투자자들이 머니 마켓 펀드에서 자금을 대규모로 인출했기 때문인데, 주로 금리 변동과 경제 불확실성에 따른 현금 수요 증가가 원인으로 분석됩니다. 머니 마켓 뮤추얼 펀드는 단기 금융상품에 투자하는 펀드로, 안전성과 유동성이 높아 단기 자금 운용에 많이 활용되지만, 최근 시장 상황 변화로 인해 자산 규모가 크게 줄어든 것입니다.
